# Rekurzija

Dobro razumevanje rekurzije se običajno izkaže za konceptualno najtežji del Programiranja 2. Uvodne naloge izgledajo lahke, slediti opisu pravilne rešitve težjih nalog pa tudi ni pretežko. Sestaviti svojo rešitev in v njej odpraviti vse napake pa zahteva veliko vaje in dobrega razumevanja, kaj se v rekurziji pravzaprav dogaja.

Pri pisanju rekurzivne funkcije si lahko pogosto pomagamo s predpostavko, da funkcija že deluje za manjše probleme. Če poskrbimo, da bo res delovala za najmanjše robne primere, bo po indukciji veljala tudi za malo večje. Zaradi tega bo delovala tudi za še večje in na koncu pravzaprav za kar vse.

## Ogrevanje

Za začetek si poglejmo nalogo s seznamom, kakršno bi najbrž že znali rešiti. Napisali bomo funkcijo `vzorec`, ki bo sprejela seznam in število `k` ter vrnila seznam z vsakim `k`-tim elementom podanega seznama. Če bi seznam razbili na glavo (prvi element) in rep (preostanek seznama), ki potrebovali še dodaten parameter, ki bi nam povedal, koliko elementov z začetka seznama je nerelevantnih. Rešitev si lahko poenostavimo tako, da vzamemo prvi element, nato pa odrežemo prvih `k` elementov in obravnavamo preostanek. Tako je vedno relevanten prvi element v seznamu.

In [1]:
def vzorec(seznam, k):
    return [seznam[0]] + vzorec(seznam[k:], k) if seznam else []

print(vzorec([2,4,6,8,10,12,14,16], 3))

[2, 8, 14]


## Rekurzivne strukture

Določene podatkovne strukture imajo same po sebi rekurzivno strukturo in so zato primerne za rekurzivno reševanje, npr. seznami in drevesa. Reševanje pa si lahko še malo otežimo, če lahko seznami vsebujejo podsezname. Napišimo funkcijo `flatten`, ki bo sprejela seznam, ki lahko vsebuje gnezdene sezname, ki lahko vsebujejo še kakšen seznam itd. Vrne pa naj enostaven seznam vseh elementov po vrsti, kot se pojavijo v gnezdenih seznamih. V rešitvi bomo vsak element seznama dodali v rešitev z izjemo tistih, ki so pravzaprav gnezdeni seznami. Te pa lahko rekurzivno zravnamo in dodamo vse tako dobljene elemente v rešitev.

In [2]:
def flatten(seznam):
    f = []
    for x in seznam:
        if isinstance(x, list): f.extend(flatten(x))
        else: f.append(x)
    return f

print(flatten([1,[2,3],[4,[5],6,[],7],8]))

[1, 2, 3, 4, 5, 6, 7, 8]


Drevesne strukture lahko opišemo s slovarji, kjer vsak ključ ustreza nekemu vozlišču, pripadajoča vrednost pa vsebuje seznam naslednikov. Lahko pa tako strukturo zgradimo z lastnimi objekti, ki se referencirajo med seboj. Konstruktor razreda `Vozlisce` sprejme vrednost vozlišča in korena levega ter desnega poddrevesa. Z metodo `print` bomo izpisali njegovo strukturo v t.i. *preorder* vrstnem redu, kjer najprej izpišemo vrednost v korenu, nato pa vsebino levega in desnega poddrevesa. Da bo izpis bolj berljiv, dodajmo še primeren zamik glede na globino vozlišča v drevesu.

In [3]:
class Vozlisce:
    def __init__(self, x, levo=None, desno=None):
        self.x = x
        self.levo, self.desno = levo, desno

    def print(self, zamik=0):
        print(" "*zamik + str(self.x))
        if self.levo: self.levo.print(zamik+2)
        if self.desno: self.desno.print(zamik+2)

drevo = Vozlisce(10,
         Vozlisce(6,
                  Vozlisce(3),
                  Vozlisce(6,
                           None,
                           Vozlisce(9))),
         Vozlisce(15,
                  Vozlisce(12),
                  Vozlisce(20)))

drevo.print()

10
  6
    3
    6
      9
  15
    12
    20


Dopolnimo razred z metodo `urejeno`, ki bo vrnila `True` ali `False` glede na to, ali gre za urejeno drevo (https://en.wikipedia.org/wiki/Binary_search_tree), za katerega velja, da so v levem poddrevesu vrednosti manjše od tiste v korenu, v desnem pa večje ali enake.

Prva ideja je, da bi za vsako vozlišče preverili, da je koren v levem poddrevesu manjši od vrednosti v vozlišču, koren v desnem poddrevesu pa večji ali enak. Vendar to ni dovolj. Če bi v zgornjem primeru zamenjali vrednost 9 z 19, bi taka strategija razglasila drevo za urejeno, čeprav to ni res.

Vrednost v vozlišču moramo primerjati z največjo vrednostjo v levem poddrevesu (ne samo s korenom) ter z najmanjšo v desnem poddrevesu. V ta namen si pripravimo metodi `levi` in `desni`, ki bosta vrnili najbolj levo oz. desno vozlišče v drevesu. V implementaciji metode `urejeno` moramo preveriti, ali sploh obstaja levo poddrevo. Če obstaja, preverimo da je samo urejeno in da je njegov najbolj desni (največji) element manjši od vrednosti v korenu. Podobno obravnavamo še desno poddrevo.

In [4]:
class Vozlisce:
    def __init__(self, x, levo=None, desno=None):
        self.x = x
        self.levo, self.desno = levo, desno

    def print(self, zamik=0):
        print(" "*zamik + str(self.x))
        if self.levo: self.levo.print(zamik+2)
        if self.desno: self.desno.print(zamik+2)

    def desni(self):
        if self.desno: return self.desno.desni()
        else: return self.x

    def levi(self):
        if self.levo: return self.levo.levi()
        else: return self.x
        
    def urejeno(self):
        ok = True
        if self.levo:
            if not self.levo.urejeno(): ok = False
            if self.levo.desni() > self.x: ok = False
        if self.desno:
            if not self.desno.urejeno(): ok = False
            if self.desno.levi() < self.x: ok = False
        return ok

drevo = Vozlisce(10,
         Vozlisce(6,
                  Vozlisce(3),
                  Vozlisce(6,
                           None,
                           Vozlisce(9))),
         Vozlisce(15,
                  Vozlisce(12),
                  Vozlisce(20)))

print(drevo.urejeno())  # 9 -> 19 ... False

True


## Kombinatorika

Poleg nekaterih podatkovnih struktur je rekurzija pogosta tudi pri raznih kombinatoričnih strukturah kot so kombinacije, permutacije itd. V teh primerih ima rekurzivno strukturo drevo delnih rešitev, s katerimi sestavljamo iskane strukture.

### Kartezični produkt

Za začetek si oglejmo kartezični produkt oz. vse nabore ("kombinacije") števil, kjer je prvo število iz prve množice, drugo iz druge, itd. Če imamo tri take množice, lahko kartezične produkte zgeneriramo s tremi gnezdenimi zankami. Težava pa nastane, če je teh množic poljubno veliko število, recimo $n$. Če želite kdaj napisati $n$ gnezdenih zank, je to dober indikator, da boste za rešitev potrebovali rekurzijo.

V Pythonu so številne kombinatorične strukture že na voljo v modulu `itertools` (https://docs.python.org/3/library/itertools.html). Če pa želimo generirati kaj posebnega (kar ni standardna kombinacija, permutacija, ...) oz. zajeti specifičnosti problema pri generiranju vseh možnosti, pa moramo vedeti, kako te stvari delujejo oz. jih znati napisati sami, da jih lahko primerno spremenimo.

In [5]:
a, b, c = [1,2,3], [4,5], [6,7]
print("for    ", [(x,y,z) for x in a for y in b for z in c])

from itertools import product
print("product", list(product(a,b,c)))

for     [(1, 4, 6), (1, 4, 7), (1, 5, 6), (1, 5, 7), (2, 4, 6), (2, 4, 7), (2, 5, 6), (2, 5, 7), (3, 4, 6), (3, 4, 7), (3, 5, 6), (3, 5, 7)]
product [(1, 4, 6), (1, 4, 7), (1, 5, 6), (1, 5, 7), (2, 4, 6), (2, 4, 7), (2, 5, 6), (2, 5, 7), (3, 4, 6), (3, 4, 7), (3, 5, 6), (3, 5, 7)]


Obstaja več rekurzivnih pristopov. Na primeru kartezičnega produkta si bomo ogledali dva oz. tri. Prvi način bo nabiranje, kjer postopoma sestavljamo kartezični produkt, na koncu pa ga izpišemo, dodamo v globalni seznam rešitev ali kaj tretjega. V drevesni strukturi klicev imamo tako vedno večje delne strukture proti listom drevesa. V ta namen potrebujemo pomožen argument `rezultat`, s katerim si klici funkcij podajajo delne strukture. Vsako vrednost iz prvega seznama poskusimo dodati k rezultatu in rekurzivno dokončamo strukturo iz preostalih seznamov.

In [6]:
def sestavi(seznami, rezultat=[]):
    if not seznami:
        print(rezultat)
    else:
        for element in seznami[0]:
            sestavi(seznami[1:], rezultat=rezultat+[element])

sestavi([a, b, c])

[1, 4, 6]
[1, 4, 7]
[1, 5, 6]
[1, 5, 7]
[2, 4, 6]
[2, 4, 7]
[2, 5, 6]
[2, 5, 7]
[3, 4, 6]
[3, 4, 7]
[3, 5, 6]
[3, 5, 7]


Rešitev z nabiranjem lahko prilagodimo, da dejansko vrača seznam rezultatov, namesto da jih izpisuje ali spreminja globalne spremenljivke. V listih drevesa klicev vrnem seznam z eno samo rešitvijo. V ostalih primerih pa samo združimo sezname rešitev, ki nastanejo iz preostanka seznamov, če dopolnimo delno zgrajen rezultat s prvim, drugim, ..., zadnjim elementom trenutnega seznama.

In [7]:
def sestavi(seznami, rezultat=[]):
    if not seznami:
        return [rezultat]
    else:
        rezultati = []
        for element in seznami[0]:
            rezultati += sestavi(seznami[1:], rezultat+[element])
        return rezultati

print(sestavi([a, b, c]))

[[1, 4, 6], [1, 4, 7], [1, 5, 6], [1, 5, 7], [2, 4, 6], [2, 4, 7], [2, 5, 6], [2, 5, 7], [3, 4, 6], [3, 4, 7], [3, 5, 6], [3, 5, 7]]


Kot zadnjo možnost si oglejmo še "čisto" rekurzivno rešitev brez pomožnih spremenljivk. Razmisliti moramo, kako nam lahko pomagajo rešitve manjšega problema (`delni`), to so kartezični produkti vseh seznamov razen prvega. Vsem delnim rezultatom moramo na začetek dodati prvi element, nato drugi element, ... Tako iz manjših rezultatov sestavimo večjega. 

Paziti moramo na robni primer, ko nam zmanjka seznamov. V tem primeru je `return []` tipična napaka. Funkcija mora vračati seznam rešitev, ki jih nato predhodni klici poskusijo razširiti v rešitev večjega problema. Pravilna rešitev je `return [[]]`, s čimer vrnemo seznam z eno prazno rešitvijo.

In [8]:
def sestavi(seznami):
    if not seznami: return [[]]
    seznam = seznami[0]
    delni = sestavi(seznami[1:])
    rezultati = []
    for element in seznam:
        for rezultat in delni:
            rezultati.append([element] + rezultat)
    return rezultati

print(sestavi([a, b, c]))

[[1, 4, 6], [1, 4, 7], [1, 5, 6], [1, 5, 7], [2, 4, 6], [2, 4, 7], [2, 5, 6], [2, 5, 7], [3, 4, 6], [3, 4, 7], [3, 5, 6], [3, 5, 7]]


### Kombinacije

Kombinacije se od kartezičnega produkta razlikujejo v tem, da izbiramo ves čas iz iste množice, hkrati pa ni pomemben vrstni red ({a,b} in {b,a} sta ista kombinacija). Kombinacije treh elementov bi lahko zopet rešili s tremi zankami; tokrat z indeksi, da izberemo prvi element, drugega nekje desno od prve izbire itd. Ko iščemo splošne kombinacije $n$ elementov iz neke večje množice, se namesto $n$ gnezdenih zank poslužimo rekurzije ali si pomagamo s funkcijo `itertools.combinations`.

In [9]:
sez = ['a','b','c','d','e']
# print([(x,y) for x in sez for y in sez])  # narobe
print("for         ", [(sez[i],sez[j],sez[k]) for i in range(len(sez)) for j in range(i+1,len(sez)) for k in range(j+1,len(sez))])

from itertools import combinations
print("combinations", list(combinations(sez, 3)))

for          [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'b', 'e'), ('a', 'c', 'd'), ('a', 'c', 'e'), ('a', 'd', 'e'), ('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'd', 'e'), ('c', 'd', 'e')]
combinations [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'b', 'e'), ('a', 'c', 'd'), ('a', 'c', 'e'), ('a', 'd', 'e'), ('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'd', 'e'), ('c', 'd', 'e')]


Reševanja bi se lahko tako kot v prejšnjih primerih lotili z nabiranjem, vendar si raje oglejmo rekurzivno rešitev brez pomožnih argumentov. Kako lahko problem sestavljanja kombinacij razbijemo na manjše podprobleme? Na več načinov. Najbolj enostaven pa je najbrž na tiste kombinacije, ki vsebujejo prvi element iz množice in na tiste, ki ga ne. V obeh primerih kot podproblem sestavljamo kombinacije iz množice brez prvega elementa (za tega smo se že odločili, kaj bo z njim). V prvem primeru iščemo kombinacije $n-1$ elementov, ki jih razširimo z dodatkom prvega elementa na začetek. V drugem pa iščemo kar kombinacije $n$ elementov, vendar iz malo manjše množice. Paziti moramo tudi, kdaj pride vsak od teh primerov sploh v poštev.

In [10]:
def kombinacije(sez, n):
    if not sez: return [[]]
    rezultati = []
    if n > 0:  # uporabi prvega
        rezultati += [sez[:1]+komb for komb in kombinacije(sez[1:], n - 1)]
    if n < len(sez):  # preskoci prvega
        rezultati += kombinacije(sez[1:], n)
    return rezultati

print("kombinacije", kombinacije(sez, 3))

kombinacije [['a', 'b', 'c'], ['a', 'b', 'd'], ['a', 'b', 'e'], ['a', 'c', 'd'], ['a', 'c', 'e'], ['a', 'd', 'e'], ['b', 'c', 'd'], ['b', 'c', 'e'], ['b', 'd', 'e'], ['c', 'd', 'e']]


### Podmnožice

Podmnožice so kombinacije poljubne velikosti. Dobimo jih lahko z unijo kombinacij velikosti od $0$ do $n$ elementov.

In [11]:
sez = [2,4,6]
subsets = []
for k in range(len(sez)+1):
    subsets += combinations(sez, k)
print("podmnozice", subsets)

podmnozice [(), (2,), (4,), (6,), (2, 4), (2, 6), (4, 6), (2, 4, 6)]


Lastna rekurzivna rešitev je pravzaprav poenostavitev rešitve za iskanje kombinacij. Samo odstranimo argument $n$, ki predstavlja velikost kombinacije, in dobimo vse podmnožice.

In [12]:
def podmnozice(sez):
    if not sez: return [[]]
    rezultati = []
    rezultati += [sez[:1]+mnoz for mnoz in podmnozice(sez[1:])]  # s prvim elementom
    rezultati += podmnozice(sez[1:])  # brez prvega
    return rezultati

sez = [2,4,6]
print("podmnozice", podmnozice(sez))

podmnozice [[2, 4, 6], [2, 4], [2, 6], [2], [4, 6], [4], [6], []]


### Permutacije

Pri permutacijah ne gre za izbor elementov, temveč za njihov vrstni red. Pomagate si lahko s funkcijo `itertools.permutations`.

In [13]:
from itertools import permutations
sez = ["a","b","c"]
print("permutations", list(permutations(sez)))

permutations [('a', 'b', 'c'), ('a', 'c', 'b'), ('b', 'a', 'c'), ('b', 'c', 'a'), ('c', 'a', 'b'), ('c', 'b', 'a')]


Pri načrtovanju lastne rešitve tudi tu obstaja več pristopov. Lahko izberemo, kateri od elementov seznama bo prvi element v permutaciji ali pa se odločimo, na katerem mestu v permutaciji bo končal prvi element iz seznama. Oglejmo si prvo možnost. Za vsak element seznama (`prvi = sez[i]`), ki lahko zaseda prvo mesto v permutaciji, moramo poiskati preostale elemente seznama (`ostali = sez[:i] + sez[i+1:]`) in iz njih sestaviti vse možne krajše permutacije, ki jih nato podaljšamo z dodatkom izbranega elementa na začetek.

In [14]:
def permutacije(sez):
    if not sez: return [[]]
    rezultati = []
    for i in range(len(sez)):
        prvi = sez[i]
        ostali = sez[:i] + sez[i+1:]
        rezultati += [[prvi]+perm for perm in permutacije(ostali)]
    return rezultati

print("permutacije", list(permutacije(sez)))

permutacije [['a', 'b', 'c'], ['a', 'c', 'b'], ['b', 'a', 'c'], ['b', 'c', 'a'], ['c', 'a', 'b'], ['c', 'b', 'a']]
